# Final Project


In [166]:
import mdptoolbox
import mdptoolbox.example
import numpy as np
import scipy.constants as sc
import itertools


## Unresolved Design Considerations

1. Would it make sense to use an even simpler radar state space model than the
   one considered in Selvi? We could use a 1D position/velocity space rather
   than 3D.


## Radar Environment


The radar environment is defined by a set of possible postion states
$\mathcal{P}$ and a set of velocity states $\mathcal{V}$.

$\mathcal{P} = \{\mathbf{r_1}, \mathbf{r_2}, \dots, \mathbf{r_\rho}\}$

$\mathcal{V} = \{\mathbf{v_1}, \mathbf{v_2}, \dots, \mathbf{v_\nu}\}$

where $\rho$ is the number of possible position states and $\nu$ is the number
of possible velocities.

Each $\mathbf{r_i}, \mathbf{v_i}$ are 3-dimensional row vectors

$\mathbf{r_i} = \left[r_x, r_y, r_z \right]$

$\mathbf{v_i} = \left[v_x, v_y, v_z \right]$

where x, y and z represent the cross-range, down-range, and vertical dimensions,
respectively.


For this simple script, only 1D range and velocity will be considered.


## Simulation Objects


In [167]:
class Radar:
    """Monostatic radar object"""

    def __init__(self, pos=np.zeros((3,)), prf=1e3, center_freq=10e9, tx_gain=100, tx_power=100, samp_rate=1e6, num_pulse_cpi=128):
        self.position = np.array(pos)
        self.prf = prf
        self.center_freq = center_freq
        self.tx_gain = tx_gain
        # Assume monostatic
        self.rx_gain = tx_gain
        self.tx_power = tx_power
        self.samp_rate = samp_rate
        self.num_pulse_cpi = num_pulse_cpi
        # Derived parameters
        self.lambda0 = sc.c / center_freq
        self.max_range = sc.c/(2*prf)
        self.max_doppler = prf/2

    def rx_power(self, target):
        Pr = self.tx_power*self.G**2*self.lam**2 * \
            target.rcs/((4*sc.pi)**3*target.range**4)
        return Pr


class Waveform:
    """Linear FM Waveform object"""

    def __init__(self, bandwidth, pulsewidth):
        self.bandwidth = bandwidth
        self.pulsewidth = pulsewidth


class Target:
    """Point target object"""

    def __init__(self, pos, vel, rcs):
        self.pos = np.array(pos)
        self.vel = np.array(vel)
        self.rcs = rcs


class Interference:
    """Abstract interference object"""

    def __init__(self):
        pass


In [168]:
# Radar system
radar = Radar(pos=np.array(0), prf=2000,
              center_freq=10e9, tx_gain=100, tx_power=100,num_pulse_cpi=20)
# Number of position states
rho = 50
r = np.linspace(0, radar.max_range, rho)
# Number of velocity states
nu = 10
v = np.arange(-radar.prf/2, radar.prf/2, radar.prf/nu)*(radar.lambda0/2)


### Interference Model


In [169]:
N = 5
channel_bw = 100e6
subband_bw = channel_bw / N
channel = np.zeros((N,))
actions = np.zeros((N,))
# Matrix of all unique interference states, where each row is a unique state and
# each column is a frequency bin
Theta = np.array(list(itertools.product([0, 1], repeat=N)))


### Reward structure


Use the reward function defined in table 2 of Selvi2020


In [170]:
def reward(sinr, bw):
    r = 0
    # SINR reward structure
    if sinr < 0:
        r -= 35
    elif sinr >= 0 and sinr <= 2:
        r += 1
    elif sinr > 2 and sinr <= 5:
        r += 2
    elif sinr > 5 and sinr <= 8:
        r += 3
    elif sinr > 8 and sinr <= 11:
        r += 4
    elif sinr > 11 and sinr <= 14:
        r += 5
    elif sinr > 14 and sinr <= 17:
        r += 6
    elif sinr > 17 and sinr <= 20:
        r += 8
    else:
        r += 10

    # Bandwidth reward structure
    r += 10*(bw-1)
    return r


## Train and test the cognitive radar

See algorithm 1 in appendix B of Selvi2020


In [171]:
# Number of possible states
S = rho*nu*2**N
# Number of possible actions
A = int(N*(N+1)/2)
# Initialize the transition and reward matrices
T = np.zeros((A, S, S))
R = np.zeros((A, S, S))

num_train = int(1e3)
num_test = int(1e2)
target = Target(pos=[], vel=[], rcs=0.1)
for itrain in range(num_train):
    # Randomly select a starting position and target velocity
    target.pos = np.random.choice(r)
    target.vel = np.random.choice(v)
    # Add a gaussian perturbance to the position and velocity
    target.pos += np.random.randn()
    target.vel += np.random.randn()
    # TODO: Calculate initial SINR
    # TODO: for each time index do
    # TODO: Calculate the initial state
    # TODO: Randomly select a valid action
    # TODO: Determine the bandwidth used, then update the interference, position, range,and SINR
    # TODO: Determine the new state
    # TODO: Update the transition and reward matrices
    pass

# TODO: Use policy iteration to determine the optimal policy
# Discount factor
gamma = 0.9
# pi = mdptoolbox.mdp.PolicyIteration(T, R, gamma)
for itest in range(num_test):
    # TODO: Select a NEW trajectory that was not used for training
    # TODO: Calculate initial SINR
    # TODO: for each time index do
    # TODO: Calculate the initial state
    # TODO: Select an action from the policy
    # TODO: Determine bandwidth used, update interference, position, range, SINR
    # TODO: Determine new state
    pass

# TODO: ???
# TODO: Profit
